In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#Modeling
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.model_selection import RandomizedSearchCV 
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [8]:
df=pd.read_csv('C:\\Project_git\\MLproject\\notebook\\data\\StudentsPerformance.csv')

In [9]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [12]:
X= df.drop(columns=['math score'],axis=1)


In [13]:
y= df['math score']

In [15]:
#create ColumnTransformer with 3 types of transformers
num_features =X.select_dtypes(exclude=['object']).columns
cat_features = X.select_dtypes(include=['object']).columns  
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer 
 
numeric_transformer = StandardScaler() 
oh_transformer = OneHotEncoder() 

preprocessor = ColumnTransformer(
    transformers=[
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features)
    ]
)



In [19]:
y.head()

0    72
1    69
2    90
3    47
4    76
Name: math score, dtype: int64

In [17]:
X.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [21]:
X = preprocessor.fit_transform(X) #preprocess the data

In [22]:
X.shape

(1000, 19)

In [24]:
#seperate the data into train and test sets 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((800, 19), (200, 19), (800,), (200,))

In [25]:
#create a function to evaluate the models

def evaluate_model(true, pred):
    mae= mean_absolute_error(true, pred)
    mse= mean_squared_error(true, pred) 
    rmse= np.sqrt(mse)
    r2_square= r2_score(true, pred)
    return mae, mse, rmse, r2_square

In [28]:
models={
    "LinearRegression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "KNeighborsRegressor": KNeighborsRegressor(),
    "DecisionTreeRegressor": DecisionTreeRegressor(),
    "RandomForestRegressor": RandomForestRegressor(),
    "AdaBoostRegressor": AdaBoostRegressor(),   
    "CatBoostRegressor": CatBoostRegressor(verbose=0),
    "XGBRegressor": XGBRegressor(),
}
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model_name = list(models.keys())[i]
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae, model_train_mse, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, _, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)


    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print("Model performance on train set")
    print("- Root Mean Squared Error:{:.4f} ".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))
    r2_list.append(model_train_r2)

    print('='*35)
    print('\n')



LinearRegression
Model performance on train set
- Root Mean Squared Error:5.3231 
- Mean Absolute Error: 4.2667
- R2 Score: 0.8743


Lasso
Model performance on train set
- Root Mean Squared Error:6.5938 
- Mean Absolute Error: 5.2063
- R2 Score: 0.8071


Ridge
Model performance on train set
- Root Mean Squared Error:5.3233 
- Mean Absolute Error: 4.2650
- R2 Score: 0.8743


KNeighborsRegressor
Model performance on train set
- Root Mean Squared Error:5.7092 
- Mean Absolute Error: 4.5180
- R2 Score: 0.8554


DecisionTreeRegressor
Model performance on train set
- Root Mean Squared Error:0.2795 
- Mean Absolute Error: 0.0187
- R2 Score: 0.9997


RandomForestRegressor
Model performance on train set
- Root Mean Squared Error:2.3098 
- Mean Absolute Error: 1.8343
- R2 Score: 0.9763


AdaBoostRegressor
Model performance on train set
- Root Mean Squared Error:5.7991 
- Mean Absolute Error: 4.7554
- R2 Score: 0.8508


CatBoostRegressor
Model performance on train set
- Root Mean Squared Error:3.

In [30]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model', 'R2 Score']).sort_values(by='R2 Score', ascending=False)

,Model,R2 Score
4,DecisionTreeRegressor,0.999653
8,XGBRegressor,0.995500
5,RandomForestRegressor,0.976335
7,CatBoostRegressor,0.958936
0,LinearRegression,0.874317
2,Ridge,0.874304
3,KNeighborsRegressor,0.855420
6,AdaBoostRegressor,0.850831
1,Lasso,0.807146


In [31]:
lin_model = LinearRegression(fit_intercept=True)
lin_model.fit(X_train, y_train)
y_train_pred = lin_model.predict(X_test)
score=r2_score(y_test, y_train_pred)*100
print("Accuracy of Linear Regression model is:%.2f " %score)

Accuracy of Linear Regression model is:88.04 


In [36]:
plt.scatter(y_test, y_test_pred)
plt.xlabel("True Values")
plt.ylabel("Predicted Values")



ImportError: cannot import name '_has_pil' from 'matplotlib.backend_bases' (c:\Project_git\MLproject\venv\lib\site-packages\matplotlib\backend_bases.py)

In [37]:
import matplotlib.pyplot as plt

plt.plot([1, 2, 3], [4, 5, 6])
plt.title("Matplotlib is working 🎉")
plt.show()


ImportError: cannot import name '_has_pil' from 'matplotlib.backend_bases' (c:\Project_git\MLproject\venv\lib\site-packages\matplotlib\backend_bases.py)